# Sumulation of product formulation design using Bayesian Optimization

### Introduction
- Bayesian optimization is a sequential design strategy for global optimisation of “black-box” functions via intelligent exploration of the parameter space.
- It builds a surrogate model for the objective and quantifies the uncertainty in that surrogate model using Gaussian process (GP) regression.
- It then employs an acquisition function defined from this surrogate model to decide where to sample. 
- Gaussian process regression is underpinned by a kernel, which is chosen such that points that are closer have a large positive correlation, encoding the belief that they should have more similar function values than points that are far apart. 
- It is customary to explore different kernels, along with their associated parameters.
- The process therefore  generates a sequence of (suggested) results/experiments as the optimization improves 

There are several libraries for Bayesian Optimization (e.g., see https://sheffieldml.github.io/GPyOpt/). This implementation was to improve understanding of the technique

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern,RationalQuadratic,RBF
from scipy.stats import norm
from scipy.optimize import minimize,  LinearConstraint
import numpy as np
import pandas as pd
import sys

import warnings
warnings.filterwarnings('ignore')

Experimentation is simulated using a plynomial function. Typically, this should be an actual formulation (i.e., experiments in the lab) which is *expensive* both in terms of time and money. 

In [ ]:

def polynomial_aux(x,y,degree):
    '''
        the polynomial function
        x - value of sles
        y - value of lauric acid
        degree - the degree of the polynomial, currently 1 - 4 
    '''
    cf	= [513.8876003,485.5712164,-131.6215742]
    cf2 = [866.6232174,668.6746113,-317.2022851,-457.7137891,-238.8347154]
    cf3 = [-6142.274035,-13688.30946,12805.9489,74782.81504,-7834.368912,-122278.7583,1773.422826]
    cf4 = [-283247.8119,76342.95751,777456.662,-609458.0096,-917419.5376,2086708.454,394646.3515,-2589721.103,34036.70983]
    if degree == 1:
        return x * cf[0] + cf[1] * y + cf[2]
    if degree == 2:
        return x * cf2[0] + y * cf2[1] + x * x * cf2[2] + y * y * cf2[3] + cf2[4]
    if degree == 3:
        return  x * cf3[0] + y * cf3[1] + x * x * cf3[2] + y * y * cf3[3] + pow(x, 3) * cf3[4] + pow(y, 3) * cf3[5] + cf3[6]
    if degree == 4:
        return x * cf4[0] + y * cf4[1] + pow(x, 2) * cf4[2] + pow(y,2) * cf4[3] + pow(x, 3) * cf4[4] + pow(y, 3) * cf4[5] +  pow(x, 4) * cf4[6] + pow(y, 4) * cf4[7] + cf4[8]  

## the main polynomial function
def polynomial(x,y,degree):
    '''
        the polynomial function
        x - value of sles
        y - value of lauric acid
        degree - the degree of the polynomial, currently 1 - 5 where 5 is the max of 3 & 4
    '''
    if degree >= 1 and degree <=4:
        return polynomial_aux(x,y,degree)
    if degree == 5:
        return max(polynomial_aux(x,y,3),polynomial_aux(x,y,4))
    else: 
        raise Exception("Sorry, no degrees index greater than 5 (which is the maximum of 3 & 4) allowed") 

Generate the data grid that underpins the heathmap

In [ ]:

def generate_datagrid(start1,end1,start2,end2,start3,end3,scale,total = 1):
    '''
    Creates a data grid of points within the sample space
    for sles, lauric acid and CAPB
    
    Parameters
    -----
    start1 : the lower bound (of sles in this case)
    end1 : the upper bound of (of sles in this case)
    start2 : the lower bound (of lauric acid in this case)
    end2 : the upper bound (of lauric acid in this case)
    start3 : the lower bound (of capb in this case)
    end3 : the upper bound (of capb in this case)
    scale : determines the number of cells in the grid (approx. scale * scale * 0.16)
    total : the constraint on the grid, which is 1 in this case
    '''
    range1 = (end1 - start1)
    range2 = (end2 - start2)
    scale1 = (scale * range1 / (range1 + range2)) 
    increment1 = (end1 - start1)/ scale1
    increment2 = (end2 - start2) / (scale - scale1)
    data_list = []
    for value1 in np.arange(start1 + increment1,end1 , increment1):
        for value2 in np.arange(start2 ,end2 , increment2):
            val = value1 + value2
            if val < total :
                capb = total - val
                if capb >= start3 and capb <= end3:
                    data = {'Sles': value1, 'LauricAcid': value2, 'CAPB' : total - value1 - value2}
                    data_list.append(data)    
    return pd.DataFrame(data_list)

Bayesian optimization processing class utilising *Expected Improvement* (EI) and a *Gussian Process* acquisition function. 
 - EI is used to select the next design point in a design space, with the objective of maximizing performance. 
 - The concept of expected improvement revolves around the idea of exploring the design space systematically and efficiently, in order to find the optimal solution.
 - It balances exploration vs exploitation while optimizing the function efficiently by maximizing the expected improvement

In [ ]:
class BayesianOptimizer():      
    '''
    A bayesian optmisation processing class

    Parameters
    ----------
    target_func : the function to be optimized (maximized)
    x_init : initial x value
    y_init : initial y value 
    n_iter : number of iterations
    gp_kernel : the kernel to be used
    dataframe : the data grid with sles,lauric acid, etc
    '''
    def __init__(self, target_func, x_init, y_init, n_iter,gp_kernel,dataframe):#
        self.x_init = x_init
        self.y_init = y_init
        self.target_func = target_func
        self.n_iter = n_iter
        self.df = dataframe
        self.gauss_pr = GaussianProcessRegressor(kernel=gp_kernel)
        self.linear_constrinats = LinearConstraint([[1, 1, 1]], [1], [1])
        self.best_samples = [ {"Out_Sles" : xs[0],"Out_LAcid" : xs[1],"Out_CAPB" : xs[2], "Foam": ys[0], "EI": 0, "Sequence": n+1} for xs,ys,n in zip(x_init, y_init,range(0,len(y_init)))]
    
    def _extend_prior_with_posterior_data(self, x,y):
        self.x_init = np.append(self.x_init, np.array([x]), axis = 0)
        self.y_init = np.append(self.y_init, np.array(y), axis = 0)
        
    def _get_expected_improvement(self, x_new):
        # Estimate the Gaussian surrogate at the new trial data point   
        mean_y_new, sigma_y_new = self.gauss_pr.predict(np.array([x_new]), return_std=True)
        sigma_y_new = sigma_y_new.reshape(-1,1)
        if sigma_y_new == 0.0:
            return 0.0 
        # calculate expected improvement    
        mean_y = self.gauss_pr.predict(self.x_init)
        max_mean_y = np.max(mean_y)
        z = (mean_y_new - max_mean_y) / sigma_y_new
        exp_imp = (mean_y_new - max_mean_y) * norm.cdf(z) + sigma_y_new * norm.pdf(z)        
        return exp_imp
        
    '''
    Given that SciPy only accepts minimisation problems, we flip the sign of the 
    objective function to effect a maximization. 
    
    The parameter is the same as in _get_expected_improvement above
    '''
    def _get_negative_expected_improvement(self, x):
        return -self._get_expected_improvement(x)
            
    def _get_next_probable_point(self):
            min_ei = float(sys.maxsize)
            x_optimal = None             
            # evaluate points from the training set           
            for x_start in [[sls,lac,capb]  for sls,lac,capb in zip(self.df['Sles'], self.df['LauricAcid'], self.df['CAPB'])]:
                response = minimize(fun=self._get_negative_expected_improvement, x0=x_start, method='SLSQP', bounds=((0.3469, 0.8), (0.1, 0.3061) , (0.1,0.4) ) , constraints=self.linear_constrinats)
                if response.fun < min_ei:
                    min_ei = response.fun
                    x_optimal = response.x       
            return x_optimal, min_ei
    
    def fit(self):
        self.gauss_pr.fit(self.x_init, self.y_init)

    def optimize(self):
        y_max_ind = np.argmax(self.y_init)
        y_max = self.y_init[y_max_ind]
        optimal_x = self.x_init[y_max_ind]
        optimal_ei = 0
        error = 0
        prev_x = self.y_init[0]
        for i in range(self.n_iter):
            self.gauss_pr.fit(self.x_init, self.y_init)
            x_next, ei = self._get_next_probable_point()
            y_next = self.target_func(np.array([[x_next[0],x_next[1] ]]))  
            self._extend_prior_with_posterior_data(x_next,[y_next])            
            if y_next[0] > y_max:
                y_max = y_next[0]
                optimal_x = x_next
                optimal_ei = ei
            if i == 0:
                #prev_x = x_next
                optimal_ei = ei
            else:
                error = np.linalg.norm(prev_x - x_next)
                prev_x = x_next 
            self.best_samples.append( {"Out_Sles" : x_next[0],"Out_LAcid" : x_next[1],"Out_CAPB" : x_next[2], "Foam": y_next[0], "EI": optimal_ei, "Sequence": len(self.y_init) } )      
        return optimal_x, y_max


Testing with some initial combinations 


In [ ]:
def response_function(xs):   
    vals = [polynomial(x[0],x[1],5)  for x in xs]
    return np.array(vals) 

def create_list_chunks(list_to_chunk,val):
    ys = []
    xs = list_to_chunk
    while len(xs) > 0:
        ys.append(xs[:val])
        xs = xs[val:]
    return ys


Initialise kernels - see https://scikit-learn.org/stable/modules/gaussian_process.html 

In [ ]:
matern_kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5)
rational_quadratic_kernel = 1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1, alpha_bounds=(1e-5, 1e4))
rbf_kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0))

Bayesian Optimization Results From 4 Experiments: 
- Process is initialised with 4 experiments for the algorithm to suggest where the 5th should be 
- The suggestion is determined to be where the maximum expected improvement would be
- I.e., for every possible input, how much is foam value expected to improve from current estimates

In [ ]:
def bayes_opt_from_num(sles, lauricAcid, foam, rotate = 0, init_val = 4, max_exp = 8,scale=80,kernel = rational_quadratic_kernel,kernel_name = "RationalQuadratic"):
    results = []    
    result_filename = 'gp_results_rot'+str(rotate) + '_from'+ str(init_val)+ '.csv'
    grid_file_name = kernel_name + '_gp_grid_results_rot'+ str(rotate) +'_from' + str(init_val) + '.csv'
    if(init_val > 0 and init_val <= len(sles)):
        inputs = [np.array(vls) for vls in create_list_chunks([[sls,lac,1-sls-lac] for sls,lac in zip(sles, lauricAcid)],init_val) ]
        outputs = [vls for vls in create_list_chunks([[fm] for fm in foam],init_val) ]
        df = generate_datagrid(0.3469, 0.8, 0.1, 0.3061, 0.1, 0.4,scale)
        ## get the maximum foam value
        max_faom = max([response_function([np.array([sls,lac])]) for sls,lac, in zip(df['Sles'], df['LauricAcid'])] )
        for x_vls,y_vals,idx in zip(inputs,outputs, range(len(inputs)) ):
            if(idx == 0):  
                #fit the initial points
                bopt = BayesianOptimizer(target_func=response_function, x_init=x_vls, y_init=y_vals, n_iter=1,gp_kernel=kernel,dataframe=df)
                bopt.fit() 
                mean_header = 'GpInitMean_From' + str(init_val) 
                sd_header = 'GpInitStd_From' + str(init_val) 
                mean_and_sigma = [bopt.gauss_pr.predict(np.array([[sls,lac,capb]]), return_std=True)  for sls,lac,capb in zip(df['Sles'], df['LauricAcid'], df['CAPB'])] 
                df[mean_header] = np.asarray([x[0] for x in  mean_and_sigma], dtype=np.float32)
                df[sd_header] = np.asarray([x[1] for x in  mean_and_sigma], dtype=np.float32)
                #---
                new_x_vls = x_vls
                new_y_vls = y_vals
                for vl in range(1,max_exp, 1):
                    sequence = init_val + vl                    
                    current_processing = {"Rotation" : rotate ,"From" : sequence ,  "Length" : len(x_vls), "Kernel ": kernel_name}
                    print('Processing: ', current_processing )
                    #--------------------------------------------------------------
                    # expected improvement from the GP fitted to the init_val data
                    #--------------------------------------------------------------
                    bopt = BayesianOptimizer(target_func=response_function, x_init=new_x_vls, y_init=new_y_vls, n_iter=1,gp_kernel=kernel,dataframe=df)
                    bopt.optimize()
                    #expected Improvment from each point in the grid
                    exp_impv_header = 'ExpectedImpv_From' + str(sequence - 1) 
                    exp_impv = [bopt._get_expected_improvement(np.array([sls,lac,capb]))  for sls,lac,capb in zip(df['Sles'], df['LauricAcid'], df['CAPB'])] 
                    df[exp_impv_header] = np.asarray([x[0] for x in  exp_impv], dtype=np.float32)
                    #-----------------------------------------------------
                    #fit the next point
                    #-----------------------------------------------------
                    new_x_vls = bopt.x_init
                    new_y_vls = bopt.y_init
                    bopt.fit()
                    bopt_mean_and_sigma = [bopt.gauss_pr.predict(np.array([[sls,lac,capb]]), return_std=True)  for sls,lac,capb in zip(df['Sles'], df['LauricAcid'], df['CAPB'])]       
                    bopt_mean_header = 'GpBoptMean_From' + str(sequence) 
                    bopt_sd_header = 'GpBoptStd_From' + str(sequence) 
                    df[bopt_mean_header] = np.asarray([x[0] for x in  bopt_mean_and_sigma], dtype=np.float32)
                    df[bopt_sd_header] = np.asarray([x[1] for x in  bopt_mean_and_sigma], dtype=np.float32)
                    results += [{**current_processing, **value} for value in bopt.best_samples ]
                    latest_foam = bopt.best_samples[-1]
                    ##  stop processing if the maximum foam value has been found
                    if(latest_foam['Foam'] >= max_faom[0]):
                        print("Maximum foam value reached in sequence " , sequence , " for rotation ", rotate)
                        break
        df.to_csv(grid_file_name, encoding='utf-8', index=False, header=True)            
    else:
        raise Exception("The init_val value must be greater than 0 and less than or equal to " + str(len(sles)) )
    result_df = pd.DataFrame(results)
    result_df.to_csv(result_filename, encoding='utf-8', index=False, header=True)

Evaluating different initial combinations

In [ ]:
orig_sles = [0.8, 0.68892858, 0.62, 0.3469, 0.483349001, 0.5, 0.67632656, 0.5938776, 0.514433498, 0.3469]
orig_lauricAcid = [0.1, 0.144428568, 0.1, 0.2531, 0.173492419, 0.1, 0.22367344, 0.3061224, 0.273877504, 0.3061224]
orig_foam = [300, 300, 310, 250, 160, 140, 300, 275, 310, 140, 300, 300, 350, 250, 170, 105, 285, 350, 320, 140, 320, 270, 300, 200, 150, 130, 320, 350, 300, 130]

## Average the 30 measured foam values 
def avg_foam (val = 3):
    return [ (orig_foam[i] + orig_foam[i + 10] + orig_foam[i + 20])/val for i in range(len(orig_sles))]
foam_n = avg_foam()

#move the first ln elements of the list ot the back
def rotate_list_at(list_to_rotate, ln):
    if (len(list_to_rotate) > ln and ln > 0):
        back = list_to_rotate[:ln]
        front = list_to_rotate[ln:]        
        return front + back
    else:         
        return list_to_rotate


#generate results for all rotations, using all kernels
def bayes_opt_for_all(rotation_limit):
    if (rotation_limit >= 0 and rotation_limit <= len(orig_sles)):
        for kernel, kernel_name in zip( [matern_kernel, rational_quadratic_kernel, rbf_kernel], ['Matern', 'RationalQuadratic', 'RBF'] ):
            for n in range(rotation_limit):
                rot_sles = rotate_list_at(orig_sles,n)
                rot_lauricAcid = rotate_list_at(orig_lauricAcid,n)
                rot_foam = rotate_list_at(foam_n,n)
                bayes_opt_from_num(sles=rot_sles,lauricAcid= rot_lauricAcid, foam=rot_foam,rotate = n,scale=135, kernel=kernel,kernel_name=kernel_name)
    else:
        raise Exception("The rotation number must be at least 0 and less than " + str(len(orig_sles)) )

In [ ]:
## run the bayesian optimisation for all rotations
bayes_opt_for_all(rotation_limit=10)